In [82]:
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import fiasco

%matplotlib inline

# Testing an Ion Collection Object
Need to have an object that combines several ions into a collection. This is most useful for calculating multi-ion quantities like spectra and radiative losses.

What are the ways we could create an `IonCollection`?

1. Instantiate an `IonCollection` object with
  - a list of ion strings, e.g. `['fe_1','ca_7','li_2']`
  - a list of `Ion` objects, e.g. `[Ion('fe_1'),Ion('ca_7'),Ion('li_2')]`
  - a list of element strings, e.g. `['Fe','Ca']`
  - a list of `Element` objects, e.g. `[Element('Fe'),Element('Ca')]`
  - any combination of the above
2. Add two or more `Element` or `Ion` objects together, e.g.
  - `Element('Fe') + Element('Ca')`
  - `Ion('fe_1') + Ion('ca_7')`
3. Add two or more `IonCollection` objects together
  
This make things like spectra and radiative losses easily "composable" and is very intuitive for the user, particularly the latter approach. 

A few notes:
- To have two or more ions in an `IonCollection`, the temperature and densities **must be the same**.
- If only a string is included anywhere in the list, a temperature must also be specified
- When an element is added to a collection, it is expanded into its component ions, i.e. there is no need to have a separate collection for elements
- We need to filter out duplicates such that if I include both `Element('Fe')` and `Ion('fe_2')`, I don't have Fe II in the collection twice


In [107]:
temperature = np.logspace(5,8,100)*u.K

In [174]:
class IonCollection(object):
    
    def __init__(self,item_list,temperature=None):
        if type(item_list) is not list:
            item_list = [item_list]
        self._ion_list = []
        for item in item_list:
            if isinstance(item, fiasco.Ion):
                self._ion_list.append(item)
            elif isinstance(item, fiasco.Element):
                self._ion_list += [ion for ion in item]
            elif isinstance(item, type(self)):
                self._ion_list += item._ion_list
            elif type(item) is str or type(item) is int:
                if temperature is None:
                    raise ValueError('Must specify a temperature in order to create an ion or element.')
                # TODO: better criteria for ion versus element here
                if type(item) is str and len(item.split('_')) > 1:
                    self._ion_list.append(fiasco.Ion(item,temperature))
                else:
                    self._ion_list += [ion for ion in fiasco.Element(item,temperature)]
            else:
                raise TypeError('{} has an unrecognized type and cannot be added to collection.'.format(item))
        # TODO: check for duplicates
        # TODO: check all temperatures are the same
        
    def __getitem__(self,x):
        return self._ion_list[x]
    
    def __contains__(self,x):
        return x in [i.ion_name for i in self._ion_list]
    
    def __add__(self,x):
        return IonCollection(self._ion_list+[x])
    
    def __radd__(self,x):
        return IonCollection([x]+self._ion_list)

Test direct instantiation

In [175]:
el1 = fiasco.Element('Fe',temperature)
el2 = fiasco.Element('ca',temperature)
ion1 = fiasco.Ion('li_2',temperature)
ion2 = fiasco.Ion('he_2',temperature)

In [176]:
ic = IonCollection([el1,el2,ion1,'h_1','He',10],temperature)

In [177]:
for ion in ic:
    print(ion.ion_name)

fe_1
fe_2
fe_3
fe_4
fe_5
fe_6
fe_7
fe_8
fe_9
fe_10
fe_11
fe_12
fe_13
fe_14
fe_15
fe_16
fe_17
fe_18
fe_19
fe_20
fe_21
fe_22
fe_23
fe_24
fe_25
fe_26
fe_27
ca_1
ca_2
ca_3
ca_4
ca_5
ca_6
ca_7
ca_8
ca_9
ca_10
ca_11
ca_12
ca_13
ca_14
ca_15
ca_16
ca_17
ca_18
ca_19
ca_20
ca_21
li_2
h_1
he_1
he_2
he_3
ne_1
ne_2
ne_3
ne_4
ne_5
ne_6
ne_7
ne_8
ne_9
ne_10
ne_11


In [178]:
'li_3' in ic

False

In [179]:
ic2 = IonCollection('fe_2',temperature=temperature)

In [180]:
ic2[0].ion_name

'fe_2'

Testing composable approach

In [181]:
class TestIon(fiasco.Ion):
    def __add__(self,x):
        return IonCollection([self.ion_name,x],temperature=self.temperature)
    
    def __radd__(self,x):
        return IonCollection([x,self.ion_name],temperature=self.temperature)

class TestElement(fiasco.Element):
    def __add__(self,x):
        return IonCollection([self.element_name,x],temperature=self.temperature)
    def __radd__(self,x):
        return IonCollection([x,self.element_name],temperature=self.temperature)

In [182]:
tion1 = TestIon('li_2',temperature)
tion2 = TestIon('he_2',temperature)
tion3 = TestIon('fe_18',temperature)

In [183]:
ic3 = tion1 + tion2 + tion3 + ic

In [184]:
for ion in ic3:
    print(ion.ion_name)

li_2
he_2
fe_18
fe_1
fe_2
fe_3
fe_4
fe_5
fe_6
fe_7
fe_8
fe_9
fe_10
fe_11
fe_12
fe_13
fe_14
fe_15
fe_16
fe_17
fe_18
fe_19
fe_20
fe_21
fe_22
fe_23
fe_24
fe_25
fe_26
fe_27
ca_1
ca_2
ca_3
ca_4
ca_5
ca_6
ca_7
ca_8
ca_9
ca_10
ca_11
ca_12
ca_13
ca_14
ca_15
ca_16
ca_17
ca_18
ca_19
ca_20
ca_21
li_2
h_1
he_1
he_2
he_3
ne_1
ne_2
ne_3
ne_4
ne_5
ne_6
ne_7
ne_8
ne_9
ne_10
ne_11
